<a href="https://colab.research.google.com/github/mikexcohen/Calculus_book/blob/main/ch06_criticalPoints_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Calculus unraveled: Intuition, Proofs, and Python**
### Mike X Cohen (sincxpress.com)
#### https://github.com/mikexcohen/calculus_book
#### Code for Chapter 3 (Critical and inflection points)

---

# About this code file:

### This notebook contains full code solutions to the exercises in this book chapter. There are many correct ways to solve the exercises; this notebook provides *a* solution, not *THE* solution. Please use this code as a starting point to continue exploring and experimenting with calculus concepts and visualizations.

## **Using the code without the book may lead to confusion or errors.**

#### This code was written in google-colab. The notebook may require some modifications if you use a different IDE.

In [ ]:
# import libraries and define global settings
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
from IPython.display import Math

from scipy.signal import find_peaks # used for grid-search

# define global figure properties used for publication
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg') # display figures in vector format
plt.rcParams.update({'font.size':14,             # font size
                     'savefig.dpi':300,          # output resolution
                     'axes.titlelocation':'left',# title location
                     'axes.spines.right':False,  # remove axis bounding box
                     'axes.spines.top':False,    # remove axis bounding box
                     'lines.linewidth':2         # increase default line thickness
                     })

# Exercise 6.1: x-axis grid resolution

In [ ]:
# need find_peaks function from scipy's signal-processing module (imported at top of script)

# "ground truth" (analytical critical points)
analytical = np.array([-2/3,0])

# initialize
resolutions = np.arange(15,201)
sse = np.zeros(len(resolutions))
dxs = np.zeros(len(resolutions))

for i,r in enumerate(resolutions):

  # x-axis grid and function
  x = np.linspace(-1.5,.7,r)
  fx = x**2 + x**3
  dxs[i] = x[1]-x[0] # store the dx for the optional plot

  # empirical difference
  df = np.diff(fx) / dxs[i]
  dfLocalMin = find_peaks(-np.abs(df))[0]

  # SSE
  sse[i] = np.log(sum((x[dfLocalMin] - analytical)**2))


# the three lines of code below generate the plot with dx on the x-axis
#plt.figure(figsize=(12,4))
#plt.plot(dxs,sse,'s-',color=[.6,.6,.6],linewidth=1,markerfacecolor=[.2,.2,.2])
#plt.gca().set(xlim=dxs[[0,-1]],xlabel='x-axis resolution (dx)',ylabel='Error to ground truth (log)')

# this code is to show the number of points on the x-axis
plt.figure(figsize=(12,4))
plt.plot(resolutions,sse,'s-',color=[.6,.6,.6],linewidth=1,markerfacecolor=[.2,.2,.2])
plt.gca().set(xlim=resolutions[[0,-1]],xlabel='$x$-axis resolution (points)',ylabel='Error to ground truth (log)')

plt.tight_layout()
plt.savefig('critpnts_ex1.png')
plt.show()

# Exercise 6.2: Find the critical points

In [ ]:
# the function
x = sym.symbols('x')
f = x**3 - 3*x

# its 1st and 2nd derivatives
df = sym.diff(f,x)
ddf = sym.diff(f,x,2)

# find critical points and inflection points by solving f'=0
critp = sym.solve(df)
inflp = sym.solve(ddf)


# print the function and derivatives
display(Math('f(x) = %s' %sym.latex(f)))
print('')
display(Math('\\frac{df}{dx} = %s' %sym.latex(df)))
print('')
display(Math('\\frac{d^2f}{dx^2} = %s' %sym.latex(ddf)))


# print the critical and inflection points
print('')
print('Critical points (x,y):')
for cx in critp:
  print(f'  ( {float(cx)}, {f.subs(x,cx)} )')
print('')

print('Inflection points (x,y):')
for ix in inflp:
  print(f'  ( {float(ix)}, {f.subs(x,ix)} )')

In [ ]:
### now for plotting

# x-axis grid for plotting
xx = np.linspace(-2,2,901)

# plot the lines
plt.figure(figsize=(10,4))
plt.plot(xx,[f.subs(x,xi) for xi in xx],'k',linewidth=3,label=r'$f(x)=%s$'%sym.latex(f))
plt.plot(xx,[df.subs(x,xi) for xi in xx],'--',color=[.6,.6,.6],label=r"$f\,'(x)=%s$"%sym.latex(df))
plt.plot(xx,[ddf.subs(x,xi) for xi in xx],':',color=[.3,.3,.3],label=r"$f\,''(x)=%s$"%sym.latex(ddf))
plt.axhline(0,color=[.8,.8,.8],linewidth=1,zorder=-5)

# draw the points
for ci in critp:
  plt.plot(ci,f.subs(x,ci),'ko',markerfacecolor='w',markersize=9)

for ii in inflp:
  plt.plot(ii,f.subs(x,ii),'ks',markerfacecolor=[.7,.7,.7],markersize=9)


# final touches
plt.legend()
plt.gca().set(xlim=xx[[0,-1]],ylim=[-10,10],xlabel='$x$',ylabel=r"$f$ or $f\,'$ or $f\,''$")

plt.tight_layout()
plt.savefig('critpnts_ex2.png')
plt.show()

# Exercise 6.3: cube roots in sympy

In [ ]:
1/3
1/sym.sympify(3)

In [ ]:
x = sym.symbols('x')
fx = x**(sym.sympify(1)/3)
sym.plot(fx,(x,-3,3));
fx.subs(x,-8)

In [ ]:
fx = sym.real_root(x,3)
sym.plot(fx);
fx.subs(x,-8)

In [ ]:
# the function and its derivative (symbolic)
x = sym.symbols('x')#,real=True)
fx = sym.real_root(x,3)
df = sym.diff(fx)

# functions for the functions
fx_fun = sym.lambdify(x,fx)
df_fun = sym.lambdify(x,df) # should work but will cause problems!

# numerical values for plotting
xx = np.linspace(-2,2,321)
df_num = df_fun(xx)
# df_num = [ df.subs(x,xi) for xi in xx ] # numerical evaluation is better

plt.figure(figsize=(5,6))
plt.plot(xx,fx_fun(xx),'k',label=r'$f(x) = %s$' %sym.latex(fx))
plt.plot(xx,df_num,'--',color=[.4,.4,.4],label=r"$f\,'(x) = %s$" %sym.latex(df))
plt.axhline(0,linestyle=':',color=[.8,.8,.8],zorder=-3)

plt.legend(loc='lower right')
plt.gca().set(xlim=xx[[0,-1]],xlabel='$x$',ylim=[-1.5,1.5],ylabel="$f$ or $f\,'$")

plt.tight_layout()
plt.savefig('critpnts_ex3.png')
plt.show()

# Exercise 6.4: cube roots in numpy

In [ ]:
# the function and its derivative (numerical)
xx = np.linspace(-2,2,321)
dx = xx[1]-xx[0]

# numerical calculation of function and its derivative
fx = xx**(1/3)
# fx = np.cbrt(xx)
df = np.diff(fx) / dx

plt.figure(figsize=(5,6))
plt.plot(xx,fx,'k',label=r'$f(x) = \sqrt[3]{x}$')
plt.plot(xx[:-1],df,'--',color=[.4,.4,.4],label=r"$f'(x) = \frac{1}{3\sqrt[3]{x^2}}$")
plt.axhline(0,linestyle=':',color=[.8,.8,.8],zorder=-3)

plt.legend()
plt.gca().set(xlim=xx[[0,-1]],xlabel='$x$',ylim=[-1.5,1.5],ylabel="$f$ or $f\,'$")

plt.tight_layout()
plt.show()

# Exercise 6.5: the butterfly :)

In [ ]:
# symbolic variables
x,alpha = sym.symbols('x,alpha')

# base equation
fx = (2*(x-alpha)**2+3) / ((x-1)**2+2)
df = sym.diff(fx,x)

# lambdify
fx_l = sym.lambdify((x,alpha),fx)
df_l = sym.lambdify((x,alpha),df)


# variables for plotting
xx = np.linspace(-5,7,401)
alphas = np.linspace(-1,3,22)
lineColors = np.linspace(.1,.9,len(alphas))

_,axs = plt.subplots(2,1,figsize=(10,8))

for i,a in enumerate(alphas):

  # evaluate the function
  # y = [ fx.subs({x:xi,alpha:a}) for xi in xx ] # FYI, this line takes a long time to compute!
  y  = fx_l(xx,a)
  dy = df_l(xx,a)

  # get the critical points
  cp = sym.solveset(df.subs(alpha,a),x,domain=sym.S.Reals)
  cp = np.array(cp.args) # overwriting the variable name
  cp = cp[ np.argmax(fx_l(cp,a)) ] # find the cp associated with maximal f(x)


  # draw the lines
  c = lineColors[i]
  axs[0].plot(xx,y,color=[c,c,c])
  axs[1].plot(xx,dy,color=[c,c,c])

  # draw the critical points
  axs[0].plot(cp,fx_l(cp,a),'ko',markersize=8,markerfacecolor=[c,c,c])


# final adjustments
axs[0].set(xlim=xx[[0,-1]],ylabel=r'$y = f(x)$',title=r'$\bf{A}$)  The function for various $\alpha$ values')
axs[1].set(xlim=xx[[0,-1]],xlabel='$x$',ylabel=r"$y\,' = f\,'(x)$",title=r'$\bf{B}$)  Derivatives for various $\alpha$ values')

plt.tight_layout()
plt.savefig('critpnts_ex5.png')
plt.show()

# Exercise 6.6: Visualizing gradient descent

In [ ]:
# function and derivative
def fx(x): return 3*x**2 - 2*x + np.pi
def df(x): return 6*x - 2

# x-axis grid
x = np.linspace(-1,2,2001)

# starting point
localmin = -1


# plot the results
plt.figure(figsize=(10,5))
plt.plot(x,fx(x),'k',label=r'$f(x) = 3x^2 - 2x + \pi$')
plt.plot(x,df(x),'--',color=[.4,.4,.4],label=r"$f'(x) = 6x-2$")

# learning parameters
learning_rate = .01
training_epochs = 100

marksizes = np.linspace(15,3,training_epochs)
markcolors = np.linspace(0,1,training_epochs)

# run through training
for i in range(training_epochs):

  # calculations
  grad = df(localmin)
  localmin = localmin - learning_rate*grad

  # plot the guesses
  if i%5==0:

    # color and size of the marker
    c = markcolors[i]
    s = marksizes[i]

    # plot this point
    plt.plot(localmin,fx(localmin),'ko',markersize=s,markerfacecolor=[c,c,c])
    plt.plot(localmin,df(localmin),'ko',markersize=s,markerfacecolor=[c,c,c])


# finish the plot
plt.gca().set(xlim=x[[0,-1]],ylim=[-10,12],xlabel='$x$',ylabel="$f$ or $f\,'$")
plt.grid(color=[.9,.9,.9])
plt.axvline(1/3,linestyle=':',color=[.3,.3,.3],zorder=-4,label='Exact minimum')
plt.legend()

plt.tight_layout()
plt.savefig('critpnts_ex6.png')
plt.show()

# Exercise 6.7:

In [ ]:
# some wacky-looking function
def fx(x):
  piece1 = np.exp(np.cos(2*x))**2
  piece2 = np.log(x**2) * (x>0) # the boolean multiplication implements the piecewise with equal vector length
  return piece1 - piece2 # OK, technically this is more than one line, lol, but you could combine the previous 2 lines


# x-axis grid
x = np.linspace(-1.5,4.9,9979)
dx = x[1]-x[0]

# calculate y and dy
y = fx(x)
y[np.argmin(abs(x))-1] = np.nan # for visualization
dy = np.diff(y) / dx
ddy = np.diff(dy) / dx

# find critical points
critMinima = find_peaks(-np.abs(dy))[0]
critPnts = []
for i in critMinima:
  if np.sign(dy[i-1]) + np.sign(dy[i+1]) == 0:
    critPnts.append(i)

# there is a critical point at x=0 at the jump discontinuity (left of zero)
critPnts.append( np.argmin(abs(x))-1 )


# identify inflection points
inflMinima = find_peaks(-abs(ddy))[0]
inflPnts = []
for i in inflMinima:
  if np.sign(ddy[i-1]) + np.sign(ddy[i+1]) == 0:
    inflPnts.append(i)



# draw the function
_,axs = plt.subplots(1,3,figsize=(14,4))
axs[0].plot(x,y,'k',label='Function')
axs[0].plot(x[critPnts],fx(x[critPnts]),'ko',markerfacecolor='w',markersize=9,label='Critical points')
axs[0].plot(x[inflPnts],fx(x[inflPnts]),'ks',markerfacecolor=[.8,.8,.8],markersize=9,label='Inflection points')
axs[0].set(xlabel='$x$',xlim=x[[0,-1]],ylabel=r'$y$')
axs[0].grid(color=[.9,.9,.9])
axs[0].legend()
axs[0].set_title(r'$\bf{A}$)  Function')

# first derivative
axs[1].plot(x[:-1],dy,'k')
axs[1].plot(x[critPnts],dy[critPnts],'ko',markerfacecolor='w',markersize=9)
axs[1].plot(0,0,'ko',markerfacecolor='w',markersize=9) # jump discontinuity
axs[1].set(xlabel='$x$',xlim=x[[0,-1]],ylim=[-15,15],ylabel=r'$dy\,/\,dx$')
axs[1].axhline(0,linestyle='--',color=[.8,.8,.8],zorder=-3,linewidth=1)
axs[1].set_title(r'$\bf{B}$)  First derivative')

# second derivative
axs[2].plot(x[:-2],ddy,'k')
axs[2].plot(x[inflPnts],ddy[inflPnts],'ks',markerfacecolor=[.8,.8,.8],markersize=9)
axs[2].set(xlabel='$x$',xlim=x[[0,-1]],ylim=[-20,20],ylabel=r'$dy\,/\,ddx$')
axs[2].axhline(0,linestyle='--',color=[.8,.8,.8],zorder=-3,linewidth=1)
axs[2].set_title(r'$\bf{C}$)  Second derivative')



plt.tight_layout()
plt.savefig('critpnts_ex7.png')
plt.show()

# Exercise 6.8:

In [ ]:
# symbolic functions
x = sym.symbols('x',real=True)

fx  = sym.cos(x) + sym.log(sym.Abs(x))
df  = sym.diff(fx,x)
ddf = sym.diff(fx,x,2)

# and print
display(Math('f(x) = %s' %sym.latex(fx)))
print('')
display(Math('\\frac{df}{dx} = %s' %sym.latex(df)))
print('')
display(Math('\\frac{d^2f}{dx^2} = %s' %sym.latex(ddf)))

In [ ]:
# find the critical points and inflection points

## cannot be solved analytically:
# critpnts = sym.solve(df)


## so we solve it numerically:

# lambdify
fx_fun = sym.lambdify(x,fx)
df_fun = sym.lambdify(x,df)
ddf_fun = sym.lambdify(x,ddf)

# x-axis grid
xx = np.linspace(-3*np.pi,3*np.pi,1000)

fx_num = fx_fun(xx)
df_num = df_fun(xx)
# ddf_num = ddf_fun(xx) # uh oh...

# must evaluate the second derivative numerically
ddf_num = np.array([ float(ddf.subs(x,xi)) for xi in xx ])


# now to find the critical points
critMinima = find_peaks(-np.abs(df_num))[0]
critPnts = []
for i in critMinima:
  if np.sign(df_num[i-1]) + np.sign(df_num[i+1]) == 0:
    critPnts.append(i)

# identify inflection points
inflMinima = find_peaks(-abs(ddf_num))[0]
inflPnts = []
for i in inflMinima:
  if np.sign(ddf_num[i-1]) + np.sign(ddf_num[i+1]) == 0:
    inflPnts.append(i)


# print the results
print('Critical points (x,y):')
for cx,cy in zip(xx[critPnts],fx_num[critPnts]):
  print(f'  ( {cx:>5.2f}, {cy:.2f} )')

print('\nInflection points (x,y):')
for cx,cy in zip(xx[inflPnts],fx_num[inflPnts]):
  print(f'  ( {cx:>5.2f}, {cy:.2f} )')

In [ ]:
# draw the plots
_,axs = plt.subplots(1,3,figsize=(15,4))
axs[0].plot(xx,fx_num,'k')
axs[0].plot(xx[critPnts],fx_num[critPnts],'ko',markerfacecolor='w',markersize=9,label='Critical points')
axs[0].plot(xx[inflPnts],fx_num[inflPnts],'ks',markerfacecolor=[.8,.8,.8],markersize=9,label='Inflection points')
axs[0].grid(color=[.7,.7,.7],linestyle='--')
axs[0].set(xlim=xx[[0,-1]],title=r'$\bf{A}$)  The function')

axs[1].plot(xx,df_num,'k')
axs[1].plot(xx[critPnts],df_num[critPnts],'ko',markerfacecolor='w',markersize=9,label='Critical points')
axs[1].axhline(0,linestyle='--',color=[.7,.7,.7],zorder=-3)
axs[1].set(xlim=xx[[0,-1]],ylim=[-7,7],title=r'$\bf{B}$)  First derivative')

axs[2].plot(xx,ddf_num,'k')
axs[2].plot(xx[inflPnts],ddf_num[inflPnts],'ks',markerfacecolor=[.8,.8,.8],markersize=9,label='Inflection points')
axs[2].axhline(0,linestyle='--',color=[.7,.7,.7],zorder=-3)
axs[2].set(xlim=xx[[0,-1]],ylim=[-4,4],title=r'$\bf{C}$)  Second derivative')

# add legend to all axes
for a in axs: a.legend()

plt.tight_layout()
plt.savefig('critpnts_ex8b.png')
plt.show()